In [1]:
# Let's use some real life data, and make the codes as generic as possible

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
import random
import pickle
from sklearn.preprocessing import StandardScaler
import os
import tensorflow as tf
from tqdm.notebook import tqdm
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import warnings
import os
from tqdm import tqdm_notebook
import random
from scipy.stats import pearsonr
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [3]:
# load the data
X_data = fetch_california_housing()['data']
y_data = fetch_california_housing()['target']

In [4]:
warnings.filterwarnings('ignore')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.20)

In [6]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16512, 8) (4128, 8) (16512,) (4128,)


In [7]:
# define a model to optimize - here we go with the model that Ved has used, we can change this later.

# Fit a model with 256 input features, 32 neurons in the hidden layer and a single output layer

def ann(hidden = 32, input_shape = 256, output_shape = 1): 
    input_layer = tf.keras.layers.Input(shape = (input_shape))
    hidden_layer = tf.keras.layers.Dense(hidden)
    hidden_output = hidden_layer(input_layer)
    pred_layer = tf.keras.layers.Dense(output_shape, activation = "relu")
    pred_output = pred_layer(hidden_output)
#     pred_output = tf.keras.layers.Activation("softmax")(pred_output)
    model = tf.keras.models.Model(input_layer, pred_output)
    return model

In [8]:
# define a function to get the initial ensembles. 

def generate_initial_ensembles(num_weights, lambda1, size_ens):
    mean_vec = np.zeros((num_weights,))
    cov_matrix = lambda1*np.identity(num_weights)
    mvn_samp = mvn(mean_vec, cov_matrix)
    return mvn_samp.rvs(size_ens)

In [9]:
# This function seems to get the final predicted values with the initial ensemble values obtained from the MVN distribution,
# for all ensembles at once

def get_targets_with_weights(batch_data, initial_ensembles, weights_ann_1, h1, size_ens): 

    # dimensions of the target
    target_dim = 1
    
    # total number of weights in the hidden layer (without the bias)
    n_hidden_1 = len(weights_ann_1[0].ravel())
    
    # use the drawn values from the initial ensembles for just the hidden weights and arange them in the shape
    # J * features(denote this by p) * hidden_neurons(this by h) - where J is the number of ensembles
    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)
    
    
    # This will have the shape (J,n, h) after the matrix multiplication - where n is the batch size, J- no.of ensembles,
    # h - no.of hidden neurons
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    # bias will have the same dimensions as the number of hidden neurons. The shape of this will be (J, 1, h) 
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)

    # the returned output will have shape (J, n, h) (The bias values (1, h) gets added to each of the (n, h) row-wise)
    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    # moving to the last layer, the number of total weights associated with the prediction layer
    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    # arange the weights from the initial ensembles in the form (J, h, target_dim)
    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, target_dim)

    # This will have the dimensions (J,n,target_dim)
    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)

    # arange the biases from the initial ensembles - will have the shape (J, 1, target_dim, and target_dim is usually 1)
    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + target_dim)].reshape(size_ens, 1, target_dim)

    # will have the shape (J, n, 1) (biases (1,1) will get added to each (n,1) row-wise)
    final_output_1 = output_1 + output_layer_bias_1
    
    # collapse the final output across the last column - will have shape (J,n)
    final_output_1 = final_output_1[:,:, 0]
    
    ## print(final_output_1.shape, initial_ensembles.shape)
    
    # stack the weights of the output - has shape (J, n) and the initial ensembles - has shape (J, n_total_weights_of_network)
    # The stack will have a shape of (J, n + n_total_weights_of_network)
    stack = np.hstack((final_output_1, initial_ensembles))

    
    return final_output_1, stack

In [10]:
# Let's see if this function is needed - very similar to the get targets with weights function we wrote, this might be 
# necessary

def forward_operation(data1, combined_ensembles , size_ens, samp_ann_params):
    # samp_ann =  ann(hidden = hidden_neurons, input_shape = 32, output_shape = 1)
    params = samp_ann_params
    initial_ensembles1 = combined_ensembles[:, :params]
    data1_out1, data1_stack1 = get_targets_with_weights(data1, initial_ensembles1,weights_ann_1, h1, size_ens = size_ens)
    return(initial_ensembles1, data1_out1)

In [11]:
# function to give the predicted values - this function might also be redundant
def get_predictions(data1, initial_ensembles): 
    _, data1_out1 = forward_operation(data1, initial_ensembles, size_ens, samp_ann_params)
    return data1_out1

In [12]:
def calculate_mu_bar_G_bar(data1, initial_ensembles):
    mu_bar = initial_ensembles.mean(0)
    _,G_u = forward_operation(data1,initial_ensembles, size_ens = size_ens, samp_ann_params=samp_ann_params)
    G_bar = (G_u.mean(0)).ravel()
    return mu_bar.reshape(-1,1), G_bar.reshape(-1,1), G_u

In [13]:
def calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u, samp_ann_params, size_ens): 
    # computes the differences all at once for all ensembles
    u_j_minus_u_bar = initial_ensembles - mu_bar.reshape(1,-1)
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    
    # now for each ensemble compute the u thing times the g thing in te formula, and we have the C_u
    c = np.zeros((samp_ann_params, G_bar.shape[0]))
    for i in range(0, size_ens): 
        c += np.kron(u_j_minus_u_bar[i, :].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    # here c/size_ens is the C_u
    return c/size_ens

In [14]:
# This follows a similar form as the C_u computation
def calculate_D_u(G_bar, G_u, size_ens): 
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    d = np.zeros((G_bar.shape[0], G_bar.shape[0]))
    for i in range(0, size_ens): 
        d += np.kron(G_u_minus_G_bar[i,:].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    return d/size_ens

In [15]:
# Get the updated ensemble 
def get_updated_ensemble(data1, initial_ensembles, y_train, gamma, size_ens):
    mu_bar, G_bar, G_u = calculate_mu_bar_G_bar(data1, initial_ensembles)
    # get C
    C = calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u, samp_ann_params, size_ens)
    # Get D
    D = calculate_D_u( G_bar, G_u, size_ens)
    # Get Tau 
    Tau = gamma * np.identity(D.shape[0])
    # Tau inverse
    Tau_inv = np.linalg.inv(Tau)
    # D and Tau
    D_plus_cov = D + Tau_inv
    # inv of above
    D_plus_cov_inv = np.linalg.inv(D_plus_cov)
    # CD tau
    mid_quant = C@D_plus_cov_inv
    # true - predicted
    # and add some random noise in here
    mean_res = np.zeros((G_u.shape[1],))
    # the variance here can be considered as a hyperparameter
    cov_res = 0.001*np.identity(G_u.shape[1])
    mvn_samp_res = mvn(mean_res, cov_res)
    G_noise = mvn_samp_res.rvs(G_u.shape[0])
    right_quant = y_train.T.flatten().reshape(1,-1) + G_noise - G_u
    # do mid times right
    mid_times_right = mid_quant@right_quant.T
    # get the updated ensemble values
    updated_ensemble = (initial_ensembles + mid_times_right.T)
    return updated_ensemble

In [16]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [17]:
def prepare_data_train(catch_train, y_train1, idxes): 
    
    data1 = catch_train[idxes,:]
    
    y_train = y_train1[idxes].reshape(-1,1)
    
    return data1, y_train 

In [18]:
def prepare_data_test(catch_test,y_test, size): 
    idxes = random.sample(range(0, catch_test.shape[0]), k = size)
    idxes = list(idxes)
    data1 = catch_test[idxes,:]
    y_test = y_test[idxes].reshape(-1,1)
    return data1, y_test

In [19]:
# define the model
samp_ann = ann(32,X_train.shape[1],1)

# generate initial ensembles
n_weights = samp_ann.count_params()
initial_ensembles_1 = generate_initial_ensembles(n_weights, 0.01, 100)

# define other quantities
gamma = 0.001
threshold = 20
batch_size = 2500
size_ens = 100
samp_ann_params = samp_ann.count_params()
weights_ann_1 = samp_ann.get_weights()
h1 = samp_ann.layers[1].output.shape[-1]
size = X_test.shape[0]

In [20]:
def get_results(idx, print_true = True):
    
    train_rmse = []
    test_rmse = []
    
    train_idxes = random.sample(range(0, X_train.shape[0]), k = X_train.shape[0])
    
    train_chunks = list(chunks(train_idxes, batch_size))
    
    best_rmse_train = 10000
    
    data1_train_all,  y_train_all = prepare_data_train(X_train, y_train, train_idxes)
    
    
    initial_ensembles = initial_ensembles_1
    patience = 0
    
    # This is whee the epoch is denoted
    for i in range(0,300):
        
        train_chunks = random.sample(train_chunks, len(train_chunks))
        
        if print_true == True:
            print("epoch number is " +str(i))
        
        for chunk in (train_chunks):
            # for train data
            data1_train, y1_train = prepare_data_train(X_train, y_train, chunk)

            initial_ensembles = get_updated_ensemble(data1_train, initial_ensembles, y1_train, gamma, size_ens = size_ens)
        
            G_u_train = get_predictions(data1_train_all, initial_ensembles)
    
            li_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[0,:].reshape(-1,1)    
            ui_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[1,:].reshape(-1,1)  
    
            width_train = ui_train - li_train
            avg_width_train = width_train.mean(0)[0]
    
            ind_train = (y_train_all >= li_train) & (y_train_all <= ui_train)
            coverage_train= ind_train.mean(0)[0]
    
            averaged_targets_train = G_u_train.mean(0).reshape(-1,1)
            rmse_train = np.sqrt(((y_train_all - averaged_targets_train)**2).mean(0))[0]
        
            pearsonr_train = pearsonr(averaged_targets_train.reshape(averaged_targets_train.shape[0],), 
                                 y_train_all.reshape(y_train_all.shape[0],))
        
            r_train = pearsonr_train.statistic
            
            # for test data
            data1_test, y1_test = prepare_data_test(X_test,y_test, size)
    
            G_u_test = get_predictions(data1_test, initial_ensembles)

    
            li_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[0,:].reshape(-1,1)     
            ui_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[1,:].reshape(-1,1)   
    
            width_test = ui_test - li_test
            avg_width_test = width_test.mean(0)[0]
    
            ind_test = (y1_test >= li_test) & (y1_test <= ui_test)
            coverage_test= ind_test.mean(0)[0]
    
            averaged_targets_test = G_u_test.mean(0).reshape(-1,1)
            rmse_test = np.sqrt(((y1_test -averaged_targets_test)**2).mean(0))[0]  
        
            pearsonr_test = pearsonr(averaged_targets_test.reshape(averaged_targets_test.shape[0],), 
                                 y1_test.reshape(y1_test.shape[0],))
        
            r_test = pearsonr_test.statistic
            
            train_rmse.append(rmse_train)
            
            test_rmse.append(rmse_test)
            
            
            if print_true == True:
                print("Training Coverage, Widths, RMSE, and Pearson R")
                print(coverage_train, avg_width_train, rmse_train, r_train)
                print("Testing Coverage, Widths, RMSE, and Pearson R")
                print(coverage_test, avg_width_test, rmse_test, r_test)
            

            if (rmse_train < best_rmse_train): 
                best_pearsonr_train = r_train 
                best_train_width_mean = avg_width_train.mean()
                best_train_width = avg_width_train
                # best_smiles_weight = w1.mean()
                best_coverage_train = coverage_train
                best_rmse_train = rmse_train
                best_pearson_r = r_test
                best_test_width = avg_width_test

                best_coverage_test = coverage_test    
                best_rmse_test = rmse_test
                patience = 0
                best_ensembles = initial_ensembles
                
                best_test_preds = averaged_targets_test
                best_li = li_test
                best_ui = ui_test
                
                best_residuals = (y_test -averaged_targets_test)
            
            else:
                patience = patience + 1
            
            if print_true == True:
                print("Patience is")
                print(patience)
                print('\n')
        
            if patience > threshold:
                print("test_coverage" + str(best_coverage_test), flush = True)
                print("test_width" + str(best_test_width), flush = True)
                print("rmse_test" + str(best_rmse_test), flush = True)
                
                return(best_train_width, best_coverage_train, best_rmse_train, best_test_width, best_coverage_test, 
           best_rmse_test, best_pearson_r, best_ensembles, train_rmse, test_rmse,  best_test_preds, 
           best_li, best_ui, best_residuals)

In [21]:
%%time
best_train_width, best_coverage_train, best_rmse_train, best_test_width, best_coverage_test, best_rmse_test, best_pearson_r, best_ensembles, train_rmse, test_rmse, \
best_test_preds, best_li, best_ui, best_residuals = get_results(0, print_true = True)

epoch number is 0
Training Coverage, Widths, RMSE, and Pearson R
1.0 264.82570919199435 13.968452104184783 0.024598727862479106
Testing Coverage, Widths, RMSE, and Pearson R
1.0 261.18355657360524 13.41285556357376 0.02359733018832278
Patience is
0


Training Coverage, Widths, RMSE, and Pearson R
1.0 261.4260388498862 13.701001127397205 0.024677917420058055
Testing Coverage, Widths, RMSE, and Pearson R
1.0 257.7896430508467 13.154220472408776 0.02370870456711903
Patience is
0


Training Coverage, Widths, RMSE, and Pearson R
1.0 261.0981545334772 13.584625145293906 0.02474700776759591
Testing Coverage, Widths, RMSE, and Pearson R
1.0 257.46688156608946 13.041398295917563 0.023797977260010796
Patience is
0


Training Coverage, Widths, RMSE, and Pearson R
1.0 261.63732174221644 12.808142713070463 0.02492085438758681
Testing Coverage, Widths, RMSE, and Pearson R
1.0 258.0033237635947 12.294595055284685 0.02403263091134553
Patience is
0


Training Coverage, Widths, RMSE, and Pearson R
1.0 2

Training Coverage, Widths, RMSE, and Pearson R
1.0 111.64325695114718 1.3526447641507915 0.27314537009080325
Testing Coverage, Widths, RMSE, and Pearson R
1.0 110.13567291988343 1.2928335453204804 0.3124865654060529
Patience is
1


Training Coverage, Widths, RMSE, and Pearson R
1.0 107.91217356870024 1.4186151472143207 0.3275024357775228
Testing Coverage, Widths, RMSE, and Pearson R
1.0 106.43238702744712 1.3618049325371635 0.3703698351196791
Patience is
2


Training Coverage, Widths, RMSE, and Pearson R
1.0 105.17475187217318 1.4573082994728606 0.3765551403631867
Testing Coverage, Widths, RMSE, and Pearson R
1.0 103.72404448196525 1.4022276951769153 0.42160937537497734
Patience is
3


Training Coverage, Widths, RMSE, and Pearson R
1.0 101.79871686639702 1.5645389101412472 0.464311010835159
Testing Coverage, Widths, RMSE, and Pearson R
1.0 100.40014526164546 1.5118843679668084 0.509295032476135
Patience is
4


Training Coverage, Widths, RMSE, and Pearson R
1.0 97.16948945834189 1.70893